In [1]:
import os
import rasterio
import numpy as np

In [2]:
# Rename files to B,G,R
maindir1 = r'G:\MScThesis\waterQualityMonitoring\Data\MLearning\ReflectanceLayers'
cases = ['withoutGF','withGF']
n = [35,38]
for k in range(2):
    subdir1 = os.path.join(maindir1, cases[k], 'France')
    
    for file1 in os.listdir(subdir1): 
        subdir2 = os.path.join(subdir1,file1)
        for file2 in os.listdir(subdir2):
            if len(file2) == n[k]:
                name = 'B.tiff'
                if 'B03' in file2:
                    name = 'G.tiff'
                elif 'B04' in file2:
                    name = 'R.tiff'
                img_temp = rasterio.open(os.path.join(subdir2, file2))
                arr_temp = img_temp.read(1)
                outputdir = os.path.join(subdir2, name)
                with rasterio.open(outputdir,'w',driver='Gtiff', width=img_temp.width, height=img_temp.height, 
                                    count=1,crs=img_temp.crs,transform=img_temp.transform, dtype='float32', nodata=9.96921e+36) as newImg:
                    newImg.write(arr_temp,1)
                    newImg.close()    
        # Clear cache
        img_temp = None
        del newImg
        # Delete old files 
        for file2 in os.listdir(subdir2): 
            if len(file2) == n[k]:
                os.remove(os.path.join(subdir2, file2))

In [3]:
# Construct Band Ratios

maindir1 = r'G:\MScThesis\waterQualityMonitoring\Data\MLearning\ReflectanceLayers'

cases = ['withoutGF','withGF']
for case in cases:
    subdir1 = os.path.join(maindir1, case, 'France')
    img_ref = rasterio.open(os.path.join(subdir1,'20190123','B.tiff')) 
    arr_ref = img_ref.read() # to be used for initializing an empty array with same dimensions
    EPSILON = 1e-10
    for file1 in os.listdir(subdir1): # file dates
        subdir2 = os.path.join(subdir1,file1)
        my_array = np.zeros((arr_ref.shape[0],arr_ref.shape[1],arr_ref.shape[2]),dtype=np.float32)
        for file2 in os.listdir(subdir2): # Combine bands into 1 nD array
            if len(file2) == 6: # to avoid confusion with ratio bands as they will be assigned to same folder
                arr_temp = rasterio.open(os.path.join(subdir2, file2)).read()
                my_array = np.append(my_array, arr_temp, axis=0)
        my_array1 = np.array(my_array[1:,:,:])    
        # New band combinations    
        names = ['B', 'G', 'R']
        L = [i for i in range(len(names))]
        for k in range(len(L)):
            primaryBand = my_array1[k]
            primaryName = names[k]
            l = L.copy()
            if k in l:
                l.remove(k)        
            for index in l:
                secondaryBand = my_array1[index]
                secondaryName = names[index]
                ratioArray = primaryBand.copy()
                for k1 in range(ratioArray.shape[0]):
                    for k2 in range(ratioArray.shape[1]):
                        if primaryBand[k1,k2] > 1e+36 or secondaryBand[k1,k2] > 1e+36:
                            ratioArray[k1,k2] = 9.96921e+36
                        elif primaryBand[k1,k2] == -99 or secondaryBand[k1,k2] == -99:
                            ratioArray[k1,k2] = -99
                        elif secondaryBand[k1,k2] == 0:
                            ratioArray[k1,k2] = primaryBand[k1,k2]/(secondaryBand[k1,k2]+EPSILON)
                        else:
                            ratioArray[k1,k2] = primaryBand[k1,k2]/secondaryBand[k1,k2]
                # Save as separate files
                outputdir = os.path.join(subdir2, primaryName+'_'+secondaryName+'.tiff')
                with rasterio.open(outputdir,'w',driver='Gtiff', width=img_ref.width, height=img_ref.height, 
                                    count=1,crs=img_ref.crs,transform=img_ref.transform, dtype='float32', nodata=9.96921e+36) as newImg:
                    newImg.write(ratioArray,1)
                    newImg.close()